# Context

* For my initial submission, I prepared the [infernce kernel](https://www.kaggle.com/ayuraj/hpa-inference) with the assumption that each cell-level labes are same as image-level label. This gave me an LB score of 0.029.
* Thus, I trained an "image-level" multi-label classifier using this [kernel](https://www.kaggle.com/ayuraj/hpa-multi-label-classification-with-tf-and-w-b). I used this [kernel](https://www.kaggle.com/ayuraj/rgb-512x512-dataset-creation-versioning-with-w-b) to prepare 256x256 and 512x512 resolution RGB images by ignoring the Endoplasmic Reticulum (Y channel). 
* I used the segmentation tool provided by the competition hosts to get cell instance segmentation mask for each image. I built this [kernel](https://www.kaggle.com/ayuraj/hpa-segmentation-mask-visualization-with-w-b) to visualize segmentation masks using W&B.
* The idea with my initial submission was to prepare a pipeline for submission based on future improvements. 

# About this Kernel

Obviously due to weak supervision image-level labels can not be blindly assigned as cell-level labels. Thus I decided to create a **cell-level** dataset of images which have only one image-level label. I will then create a dataset of images which have two image-lebel label (will talk about it later). 

* I am using the provided segmentation tool to get cell-level masks. The tool gives cell-level mask along with nucleus-level mask. 
* I am discarding the cells whose nucleus is bordering the edge of the image. I am also discarding the cells whose nucleus could not be segmented.
* For each cell I am assigning it the image-level label. In this case since the whole image is associated to a single label, this assumption holds to some extent (we will see that after submission). 
* I also think that this dataset can be used in a weakly supervised classifier training (will explore that soon and share with you all). 

❗ **Note:** I have used this [kernel](https://www.kaggle.com/ayuraj/dataset-versioning-for-single-label-dataset) to create a `.csv` file of image_ids along with labels with only only image-level labels. You will notice that I have created 4 splits of this CSV file. This is so that I can use 9 hr limit of Kaggle kernel to create this cell-level dataset. 

## Why use Weights and Biases's Dataset and Visualization (DSViz) tool?

The dataset creation process in Kaggle can be made more lively with the use of DSViz. W&B's DSViz is a powerful new feature (still in early phase) on top of W&B Artifacts to let you visualize and query datasets and model evaluations at the example level. Check out the official doc [here](https://docs.wandb.ai/datasets-and-predictions).

It's just a few lines of code that can enable you to visualize and query your image dataset. In my opinion this makes the dataset creation pipeline more interactive. One can easily check if the generated images are correct. 

![img](https://i.imgur.com/3p9R90J.gif)

# Datasets

I have processed 3 out of 4 splits. Here's the links to to the dataset and respective W&B DSViz page:
* [HPA: Single Label RGB+Protein Dataset (0)](https://www.kaggle.com/ayuraj/singlelabel128x128split0) | [DSViz Page](https://wandb.ai/ayush-thakur/hpa/artifacts/dataset/single_label_split_0/34b460a9c6ea265a4e95/files/dataset.table.json)
* [HPA: Single Label RGB+Protein Dataset (1)](https://www.kaggle.com/ayuraj/singlelabel128x128split1) | [DSViz Page](https://wandb.ai/ayush-thakur/hpa/artifacts/dataset/single_label_split_1/b0f0ffa4613ed7ec454e/files/dataset.table.json)
* [HPA: Single Label RGB+Protein Dataset (2)](https://www.kaggle.com/ayuraj/singlelabel128x128split2) | [DSViz Page](https://wandb.ai/ayush-thakur/hpa/artifacts/dataset/single_label_split_2/584d4f0d963478a01a6e/files/dataset.table.json)

## Upcoming:

* I will combine all the images from the dataset in a single Kaggle Dataset.

# Imports and Setups

In [ ]:
%%capture

# Install W&B for ease of life.
!pip install wandb --upgrade

# Install HPA Cell Segmentation tool.
!pip install https://github.com/CellProfiling/HPA-Cell-Segmentation/archive/master.zip

In [ ]:
# General imports.
import os
import re
import cv2
import glob
import imageio
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from skimage.transform import resize
from sklearn.model_selection import train_test_split

%matplotlib inline

# HPA Segmentation tool related imports
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei

In [ ]:
# W&B import and login
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb_api")

wandb.login(key=wandb_api)

In [ ]:
# Hyperparameters
WORKING_DIR_PATH = '../input/hpa-single-cell-image-classification/'
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128

# The original csv file of image_ids with only one image-level label takes more than 9 hrs to process. 
# To be able to use Kaggle's kernel to generate the dataset I made 4 splits of the original csv.
# The SPLIT number is the index of the split used.
SPLIT = 3

In [ ]:
# Labels
# Ref: https://www.kaggle.com/divyanshuusingh/eda-image-segmentation
label_names= {
0: "Nucleoplasm",
1: "Nuclear membrane",
2: "Nucleoli",
3: "Nucleoli fibrillar center",
4: "Nuclear speckles",
5: "Nuclear bodies",
6: "Endoplasmic reticulum",
7: "Golgi apparatus",
8: "Intermediate filaments",
9: "Actin filaments",
10: "Microtubules",
11: "Mitotic spindle",
12: "Centrosome",
13: "Plasma membrane",
14: "Mitochondria",
15: "Aggresome",
16: "Cytosol",
17: "Vesicles and punctate cytosolic patterns",
18: "Negative"
}

# Image-Level Labels Analysis

Refer to this Kernel: https://www.kaggle.com/ayuraj/dataset-versioning-for-single-label-dataset

In [ ]:
# Downloading csv files.
run = wandb.init(entity='ayush-thakur', project='hpa', job_type='consume_single_label_dataset')
artifact = run.use_artifact('ayush-thakur/hpa/single_label_split:v0', type='dataset')
artifact_dir = artifact.download()
run.finish()

In [ ]:
os.listdir(artifact_dir)

In [ ]:
df = pd.read_csv(artifact_dir+f'/train_single_{SPLIT}.csv')
print(f'Total number of images: {len(df)}')
df.head()

# Train Dataset Paths

In [ ]:
red_images = sorted(glob.glob(WORKING_DIR_PATH+'train/*_red.png'))
green_images = sorted(glob.glob(WORKING_DIR_PATH+'train/*_green.png'))
blue_images = sorted(glob.glob(WORKING_DIR_PATH+'train/*_blue.png'))
yellow_images = sorted(glob.glob(WORKING_DIR_PATH+'train/*_yellow.png'))

print(len(red_images), len(green_images), len(blue_images), len(yellow_images))

In [ ]:
mt = [] # red
er = [] # yellow
nu = [] # blue
pr = [] # green

# Test if the image ids are aligned properly
for r, g, b, y in zip(red_images, green_images, blue_images, yellow_images):
    if re.findall(r'[^\/]+(?=\_.)', r)[0] == re.findall(r'[^\/]+(?=\_.)', g)[0] == re.findall(r'[^\/]+(?=\_.)', b)[0] == re.findall(r'[^\/]+(?=\_.)', y)[0]:
        if re.findall(r'[^\/]+(?=\_.)', r)[0] in df.ID.values:
            mt.append(r)
            pr.append(g)
            nu.append(b)
            er.append(y)
    else:
        print(r)
        
print(len(mt), len(pr), len(nu), len(er))

# Segment Cell and Extract Cell Level Images

In [ ]:
NUC_MODEL = "./nuclei-model.pth"
CELL_MODEL = "./cell-model.pth"

segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    device="cuda",
    padding=True,
    multi_channel_model=True,
)

# Visualize - understand the process

In [ ]:
i = 3
# Get red, blue and yellow channel images. 
microtubule = np.array(Image.open(mt[i]))
endoplasmicrec = np.array(Image.open(er[i]))
nuclei = np.array(Image.open(nu[i]))
protein = np.array(Image.open(pr[i]))
# Stack the channels to form image.
image = np.dstack((microtubule, endoplasmicrec, nuclei))

plt.figure(figsize=(5, 5));
plt.imshow(image);
plt.title(f'{df.Label.values[i]}');

In [ ]:
plt.figure(figsize=(5, 5));
plt.imshow(protein, cmap='gray');

In [ ]:
# For nuclei segmentation only blue channel is required.
nuc_segmentation = segmentator.pred_nuclei([nu[i]])
# For full cells all the three reference(except green) channels are required.
cell_segmentation = segmentator.pred_cells([[mt[i]], [er[i]], [nu[i]]])
# get cell mask
nuclei_mask, cell_mask = label_cell(nuc_segmentation[0], cell_segmentation[0])

plt.figure(figsize=(5,5))
plt.imshow(cell_mask);

In [ ]:
# number of cells
cells = np.unique(cell_mask)
print(f"Number of cells: {len(cells)}")

# visualize one cell
mask = np.where(cell_mask==1, 1,0).astype('uint8')
plt.figure(figsize=(5,5))
plt.imshow(mask);

In [ ]:
# number of cells
nucleies = np.unique(nuclei_mask)
print(f"Number of cells: {len(nucleies)}")

# visualize one cell
n_mask = np.where(nuclei_mask==1, 1,0).astype('uint8')
plt.figure(figsize=(5,5))
plt.imshow(n_mask);

In [ ]:
# get contour using mask for one cell.
cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# visualize contour
im = image.copy()
im = cv2.drawContours(im, cnts, -1, (255, 0, 255), 20)
plt.figure(figsize=(5,5))
plt.imshow(im);

In [ ]:
# get contour using mask for one nuclei.
n_cnts, _ = cv2.findContours(n_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# visualize contour
im = image.copy()
im = cv2.drawContours(im, n_cnts, -1, (255, 0, 255), 20)
plt.figure(figsize=(5,5))
plt.imshow(im);

In [ ]:
# check if the contour for nuclie is touching the image boundary. 
def is_border_nuclei(contour_points):
    unique_points = np.unique(contour_points)
    # basically if any point is 0 that means its touching the edge of the image.
    if 0 in unique_points:
        return True
    return False

is_border_nuclei(n_cnts[0])

In [ ]:
# Mask input image with binary mask
cell = cv2.bitwise_and(image, image, mask=mask)
# Color background white
cell[mask==0] = 255

# Visualize
plt.figure(figsize=(5,5))
plt.imshow(cell);

In [ ]:
# get bounding box covering the contour. 
x,y,w,h = cv2.boundingRect(cnts[0])

a = cell[y:y+h, x:x+w]
a = cv2.resize(a, (IMAGE_HEIGHT, IMAGE_WIDTH), cv2.INTER_AREA)
plt.imshow(a);

In [ ]:
# mask protein image 
cell_protein = cv2.bitwise_and(protein, protein, mask=mask)

# Visualize
plt.figure(figsize=(5,5))
plt.imshow(cell_protein, cmap='gray');

In [ ]:
a_protein = cell_protein[y:y+h, x:x+w]
a_protein = cv2.resize(a_protein, (IMAGE_HEIGHT, IMAGE_WIDTH), cv2.INTER_AREA)

plt.imshow(a_protein, cmap='gray');

# Dataset Creation Loop

In [ ]:
SAVE_DIR = f'/kaggle/tmp/hpa_single_label_128x128_split_{SPLIT}/'

os.makedirs(SAVE_DIR+'rgb', exist_ok=True)
os.makedirs(SAVE_DIR+'protein', exist_ok=True)

!ls /kaggle/tmp/hpa_single_label_128x128_split_3/

In [ ]:
# Intialize W&B run
run = wandb.init(entity='ayush-thakur', project='hpa', job_type=f'dataset-creation-single-label_split_{SPLIT}')
# Using `single_label.csv` file
artifact_csv = run.use_artifact('ayush-thakur/hpa/single_label_split:v0', type='dataset')

# Create W&B Table
table = wandb.Table(['image_id', 'image_cell', 'image_protein', 'label'])

WAND_LOG_AT = 150
NUM_IMGS = 0

for i in tqdm(range(len(df))):
    # Get image ID
    image_id = re.findall(r'[^\/]+(?=\_.)', mt[i])[0]
    
    # LOAD IMAGES
    
    # Get all channel images.
    microtubule = np.array(Image.open(mt[i]))
    endoplasmicrec = np.array(Image.open(er[i]))
    nuclei = np.array(Image.open(nu[i]))
    protein = np.array(Image.open(pr[i]))
    
    # Stack the channels to form image. Using Red, Yellow and Blue.
    image = np.dstack((microtubule, endoplasmicrec, nuclei))

    # PERFORM SEGMENTATION
    
    # For nuclei segmentation only blue channel is required.
    nuc_segmentation = segmentator.pred_nuclei([nu[i]])
    # For full cells all the three reference(except green) channels are required.
    cell_segmentation = segmentator.pred_cells([[mt[i]], [er[i]], [nu[i]]])
    # get cell mask
    nuclei_mask, cell_mask = label_cell(nuc_segmentation[0], cell_segmentation[0])

    # GET IMDIVIDUAL CELLS
    
    # Count the number of cells.
    cells = np.unique(cell_mask)
    nuclei = np.unique(nuclei_mask)

    cell_count = 0
    for cell_index in cells[1:]:
        # Get cell and nucleus mask for one cell.
        single_cell_mask = np.where(cell_mask==cell_index, 1,0).astype('uint8')
        if cell_index in nuclei:
            nucleus_mask = np.where(nuclei_mask==cell_index, 1,0).astype('uint8')
        else:
            continue
        
        # get contour for cell and nucleus
        cell_cnts, _ = cv2.findContours(single_cell_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        nucleus_cnts, _ = cv2.findContours(nucleus_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Check if the nucleus is touching the boundary of image.
        if not is_border_nuclei(nucleus_cnts[0]): # If not touching the boundary 
            # Mask the cell to be cropped.
            cell = cv2.bitwise_and(image, image, mask=single_cell_mask)
            # Mask the protein to be cropped.
            cell_protein = cv2.bitwise_and(protein, protein, mask=nucleus_mask)
            
            # Get bounding box covering the contour. 
            x,y,w,h = cv2.boundingRect(cell_cnts[0])
            # Crop cell
            cell = cell[y:y+h, x:x+w]
            # Resize cell
            cell = cv2.resize(cell, (IMAGE_HEIGHT, IMAGE_WIDTH), cv2.INTER_AREA)
            
            # Crop protein
            cell_protein = cell_protein[y:y+h, x:x+w]
            # Resize protein
            cell_protein = cv2.resize(cell_protein, (IMAGE_HEIGHT, IMAGE_WIDTH), cv2.INTER_AREA)
            
            # Save images
            cv2.imwrite(SAVE_DIR+'rgb/'+image_id+f'_{cell_count}.png', cell)
            cv2.imwrite(SAVE_DIR+'protein/'+image_id+f'_{cell_count}.png', cell_protein)
            
            # Logging every 200th generated image.
            if NUM_IMGS%WAND_LOG_AT==0:
                table.add_data(
                    image_id+f'_{cell_count}',
                    wandb.Image(cell), 
                    wandb.Image(cell_protein),
                    df.loc[df['ID']==image_id]['Label'].values[0]
                )
            
            cell_count+=1
            NUM_IMGS+=1
       
# LOG AS ARTIFACT

# Put the table in an artifact and save it
dataset_artifact = wandb.Artifact(f'single_label_split_{SPLIT}', type='dataset')
dataset_artifact.add(table, 'dataset')
run.log_artifact(dataset_artifact)

# Close W&B run
run.finish()

In [ ]:
# Copy Kaggle API token to ~/.kaggle
! mkdir -p /root/.kaggle/
! cp ../input/apitoken/kaggle.json /root/.kaggle/kaggle.json
# Initialize dataset creation
! kaggle datasets init -p /kaggle/tmp/hpa_single_label_128x128_split_3/

In [ ]:
!ls /kaggle/tmp/hpa_single_label_128x128_split_3/

In [ ]:
%%bash
echo "{
  \"title\": \"HPA: Single Label RGB+Protein Dataset (3)\",
  \"id\": \"ayuraj/SingleLabel128x128Split3\",
  \"licenses\": [
    {
      \"name\": \"CC0-1.0\"
    }
  ]
}" > /kaggle/tmp/hpa_single_label_128x128_split_3/dataset-metadata.json

In [ ]:
!kaggle datasets create -p /kaggle/tmp/hpa_single_label_128x128_split_3/ -u --dir-mode tar
# !kaggle datasets version -p /kaggle/tmp/hpa_single_label_128x128_split_0 -m "add rgb images"  --dir-mode tar